## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

In [1]:
#load libraries

from google.colab import drive
import os
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np
from glob import glob
import seaborn as sns
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
#establish connection

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load Dataset Files

Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = "/content/drive/My Drive/AIML/dog_breed/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

In [8]:
!ls 

drive  labels.csv  sample_data	sample_submission.csv  test  train


After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
label_df=pd.read_csv('labels.csv')

In [10]:
label_df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


### Print the count of each category of Dogs given in the dataset



In [11]:
label_df.groupby(['breed']).agg(['count'])

,id
,count
breed,
affenpinscher,80
afghan_hound,116
african_hunting_dog,86
airedale,107
american_staffordshire_terrier,74
appenzeller,78
australian_terrier,102
basenji,110


### Get one-hot encodings of labels

In [0]:
target=pd.Series(label_df['breed'])

In [0]:
one_hot=pd.get_dummies(target,sparse=True)
target=np.asarray(one_hot)

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [0]:
from tqdm import tqdm
im_size=100 #----> image size
ids=label_df['id']
X_train=[]
y_train=target
#print(y_train[1].shape)

for id in ids:
    #img=cv2.imread('/content/drive/My Drive/AIML/dog_breed/train/{}'.format(id)+'.jpg')
    img = cv2.imread('./train/{}.jpg'.format(id))
    X_train.append(cv2.resize(img,(im_size,im_size)))
    
X_train=np.array(X_train,np.float32)

In [15]:
#X_train = X_train[:, np.newaxis]



X_train.shape

(10222, 100, 100, 3)

Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
def standardize(array):
    array/=255
    return array
X_train=standardize(X_train)

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:
from sklearn.model_selection import train_test_split
train_x,validation_x,train_y,validation_y=train_test_split(X_train,y_train,test_size=0.3,random_state=0)

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
test_df=pd.read_csv('sample_submission.csv')

test_img=test_df['id']



Run the below code to load the test image files in x_test_feature

In [19]:

img_rows=100
img_cols=100
x_test_feature = []
i = 0 # initialisation
for f in tqdm(test_img.values): # f for format ,jpg
    img = cv2.imread('./test/{}.jpg'.format(f))
    img_resize = cv2.resize(img, (img_rows, img_cols)) 
    x_test_feature.append(img_resize)

100%|██████████| 10357/10357 [00:25<00:00, 406.02it/s]


Normalize the test data and convert it into 4 dimensions

In [20]:

x_test_feature=np.array(x_test_feature,np.float32)


x_test_feature.shape





(10357, 100, 100, 3)

In [0]:
x_test_feature=standardize(x_test_feature)



In [30]:
print (X_train.shape,x_test_feature.shape)

(10222, 100, 100, 3) (10357, 100, 100, 3)


### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [0]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalMaxPooling2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

def mode_architecture():
    model=Sequential()
    model.add(Conv2D(32,(3,3),input_shape=X_train.shape[1:],activation='relu',padding='same'))
    model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D(pool_size=(2,2),strides=2))
    #model.add(Dropout(0.20))
    
    model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
    model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D(pool_size=(2,2),strides=2))
    #model.add(Dropout(0.25))
    
    model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
    model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D(pool_size=(2,2),strides=2))
    
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(256))
    model.add(Activation('relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model



          

In [0]:
arch_cnn=mode_architecture()

In [47]:
arch_cnn.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 100, 100, 32)      896       
_________________________________________________________________
batch_normalization_11 (Batc (None, 100, 100, 32)      128       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 50, 50, 64)        18496     
_________________________________________________________________
batch_normalization_12 (Batc (None, 50, 50, 64)        256       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 25, 25, 128)       73856     
__________

### Use batch_size = 128 and epochs = 10 and execute the model

In [49]:
batch_size=128 #Randomly
epochs=10  #Randomly
print (train_x.shape,train_y.shape)

val_arch=arch_cnn.fit(X_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1)

(7155, 100, 100, 3) (7155, 120)
Epoch 1/10
10222/10222 [==============================] - 16s 2ms/step - loss: 4.9579 - acc: 0.0141
Epoch 2/10
10222/10222 [==============================] - 13s 1ms/step - loss: 4.6292 - acc: 0.0293
Epoch 3/10
10222/10222 [==============================] - 13s 1ms/step - loss: 4.4040 - acc: 0.0500
Epoch 4/10
10222/10222 [==============================] - 13s 1ms/step - loss: 4.1594 - acc: 0.0763
Epoch 5/10
10222/10222 [==============================] - 13s 1ms/step - loss: 3.8520 - acc: 0.1079
Epoch 6/10
10222/10222 [==============================] - 13s 1ms/step - loss: 3.4141 - acc: 0.1793
Epoch 7/10
10222/10222 [==============================] - 13s 1ms/step - loss: 2.6704 - acc: 0.3183
Epoch 8/10
10222/10222 [==============================] - 13s 1ms/step - loss: 1.6847 - acc: 0.5447
Epoch 9/10
10222/10222 [==============================] - 13s 1ms/step - loss: 0.8004 - acc: 0.7811
Epoch 10/10
10222/10222 [==============================] - 13s 1ms/s

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [0]:
datagen.fit(X_train)

In [0]:
arch_cnn=mode_architecture()

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:
model.fit_generator(datagen.flow(X_train, y_train, batch_size=128),steps_per_epoch=len(X_train) / 32, epochs=epochs)

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

Print the summary of the base_model

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

### Make all the layers in the base_model (VGG16) to be non-trainable

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%